# Feedparser Experimentation

In [ ]:
import numpy as np

In [ ]:
t = [50, 3, 2, 21, 34]

In [ ]:
softmax(t)

In [ ]:
t

In [ ]:
def softmax(l):
    l_exp = np.exp(l)
    sum_l_exp = np.sum(l_exp)
    for idx, i in enumerate(l_exp):
        l[idx] = i / sum_l_exp

In [ ]:
import feedparser

feed = feedparser.parse()
feed

In [ ]:
response = requests.get("http://feeds.finra.org/MonthlyDisciplinaryActions")

In [ ]:
feedparser.parse(response.text)['entries']

In [ ]:
feed['entries']

In [ ]:
titles = [item['title'] for item in feed['entries']]
titles

In [ ]:
type(feed)

In [ ]:
feed.get('etag')

In [ ]:
feed.get('e')


# Mongo Examination

In [ ]:
client = MongoClient()
coll = client['negative_news']['rss_feed']

In [ ]:
df = pd.DataFrame(list(coll.find({})))

In [ ]:
df[["Morgan" in title for title in df['content']]]

In [ ]:
import re

from dateparser.search import search_dates

In [ ]:
from pymongo import MongoClient
import pandas as pd

pd.set_option('max_colwidth', 100)

client = MongoClient("mongodb://172.24.0.4:27017")
coll = client['negative_news']['scrape_feed']
data = list(coll.find({'date': {"$in":['', None]}}))

df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
from pymongo import MongoClient
import pandas as pd

sources = [
    'Law360',
]

entity_names = [
    'Wedbush Securities',
]

pd.set_option('max_colwidth', 100)

client = MongoClient("mongodb://172.24.0.4:27017")
# coll = client['negative_news']['scrape_feed']
coll = client['negative_news']['scrape_feed']
data = list(coll.find({'entity': {'$in': entity_names}}))

data = [item for item in data if any(src.lower() in item['url'].lower() for src in sources)]
# lubin_data = [item for item in data if any(src in item['url'] for src in sources) and any(name in item['entity'] for name in entity_names)]
# lubin_data = [item for item in data if any(_id == item['jobname'] for _id in job_ids)]
orig_df = pd.DataFrame(data)

In [ ]:
from pymongo import MongoClient

entity_names = [
    'Brighthouse Life Financial',
]

client = MongoClient("mongodb://172.24.0.2:27017")
coll = client['negative_news']['scrape_feed']
coll.delete_many({'entity': {'$in': entity_names}})

In [ ]:
len(orig_df)

In [ ]:
orig_df

In [ ]:
import dateparser
from datetime import datetime

def date_extract_pandas_wrapper(row):
    # logger.debug('Extracting date: initial value is "{}" with type "{}"'.format(row['date'], type(row['date'])))

    if isinstance(row["date"], str):
        extracted_date = dateparser.parse(row["date"])
        if extracted_date is not None:
            # logger.debug('Dateparser extracted the following date: {}'.format(extracted_date))
            return datetime(extracted_date.year, extracted_date.month, extracted_date.day)
    elif isinstance(row["date"], datetime):
        return row["date"]
    elif isinstance(row["date"], pd.Timestamp):
        return row["date"].to_pydatetime()
    else:
        return None
    
def date_filter_pandas_wrapper(date, datetime_threshold):
    try:
        return date >= datetime_threshold
    except TypeError:
        return datetime(date.year, date.month, date.day, tzinfo=timezone.utc) >= datetime_threshold

df = orig_df

print('null content:')
print(len(df))
df = df.dropna(subset=["content"])
print(len(df))

print('null date:')
print(len(df))
df = df.dropna(subset=["date"])
print(len(df))

print('date threshold:')
min_year = 2018
min_month = 12

try:
    date_threshold = datetime(min_year, min_month, 1)
except ValueError:
    date_threshold = None

print(len(df))
df = df[df.apply(lambda x: date_filter_pandas_wrapper(x["date"], date_threshold), axis=1)]
print(len(df))

df

In [ ]:
df['_id'].values

In [ ]:
%%timeit
bool_list = [_id not in df['_id'].values for _id in orig_df['_id'].values]
#print(bool_list)
dropped_urls = orig_df[bool_list]['url']
for url in dropped_urls:
    #print(url)
    pass

In [ ]:
%%timeit
bool_list = [_id not in list(df['_id']) for _id in orig_df['_id']]
#print(bool_list)
dropped_urls = orig_df[bool_list]['url']
for url in dropped_urls:
    #print(url)
    pass

In [ ]:
coll1 = client['negative_news']['scrape_feed']
coll2 = client['negative_news']['processed_results']
coll1.delete_many({'entity': {'$in': entity_names}})
coll2.delete_many({'entity': {'$in': entity_names}})

In [ ]:
df['date'].value_counts()

In [ ]:
df[['date', 'url', 'scraper', 'title', 'content']]

# Date Parsing

In [ ]:
search_dates(re.sub("[()]", "", "tomorrow"), settings={"RETURN_AS_TIMEZONE_AWARE": False})

In [ ]:
df['date'] = df.apply(lambda row: min([item[1] for item in search_dates(re.sub("[()]", "", row['content'][:100]), settings={"RETURN_AS_TIMEZONE_AWARE": False})]), axis=1)

# Miscellaneous

In [ ]:
from nltk import edit_distance

In [ ]:
edit_distance("Wells Fargo", "WELLS Fargo")

# New fine extraction

In [ ]:
from nltk.stem.snowball import SnowballStemmer

RELEVANT_STEMS = "fine penalize penalty levy pay paid repay repaid assess reimburse settlement".split()
stemmer = SnowballStemmer('english')

In [ ]:
def fine_stem_exists(text, stem_list, stemmer=None):
    if stemmer is not None:
        stem_list = [stemmer.stem(stem) for stem in stem_list]
        
        
def entity_name_exists(text, entity_name_list):
    pass


def find_money_text(text):
    pass


def transform_money_tagged_entity(text):
    pass

In [ ]:
stemmer.stem("this is a test sentence")

In [ ]:
[stemmer.stem(stem) for stem in "this is a test sentence".split()]

In [ ]:
'wells' in 'Wells Fargo'.lower()

In [ ]:
test = "Wells Fargo fined $575 million by all 50 states ... Wells Fargo's problems erupted in September 2016 when the bank paid a $185 million settlement to regulators and a city official over sales ..."

from nltk.tokenize import sent_tokenize

In [ ]:
sent_tokenize(test)[0].split("...")

# PDF Parsing

In [ ]:
import io
import re

import PyPDF2
import requests


def extract_pdf_pages(fname=None, response=None):
    if fname is None:
        open_file = io.BytesIO
        args = [response.content]
    elif response is None:
        open_file = open
        args = [fname, "rb"]
    else:
        raise ValueError("Either 'fname' or 'response' must be provided")

    with open_file(*args) as f:
        pdf = PyPDF2.PdfFileReader(f)
        pdf_pages = [p.extractText() for p in pdf.pages]
    
    return pdf_pages


def inspect_pdf_obj(fname):
    with open(fname, "rb") as f:
        pdf = PyPDF2.PdfFileReader(f)
        pdf._decryption_key = b""
        pdf._override_encryption = True
        pdf._flatten()
        print(pdf.getPage(0).extractText())

response = requests.get("http://www.finra.org/sites/default/files/publication_file/Disciplinary_Actions_February_2019_0.pdf")
# raw_pdf_pages = extract_pdf_pages(response=response)
# obj = inspect_pdf_obj("firm_33135.pdf")

raw_pdf_pages = extract_pdf_pages(fname="Disciplinary_Actions_February_2019_0.pdf")

In [ ]:
import regex
from six.moves.urllib.request import urlopen
import io
import re
import logging

import pdftotext

logger = logging.getLogger(__name__)

debug = True

REQUIRED_KEYS = [
    "Reporting Source",
    "Current Status",
    "Allegations",
    "Initiated By",
    "Date Initiated",
    "Docket/Case Number",
    "Principal Product Type",
    "Other Product Type(s)",
    "Principal Sanction(s)/Relief Sought",
    "Other Sanction(s)/Relief Sought",
    "Resolution Date",
    "Resolution",
    "Other Sanctions Ordered",
    "Sanctions Ordered",
    "Sanction Details",
]

OPTIONAL_KEY_DICT = {
    "Final Order Flag": "No",
    "URL for Regulatory Action": "",
}

IGNORE_ENSURE_NEW_LINE = [
    "Sanctions Ordered"
]

def parse_finra_broker_check_report(crd):
    
    pdf_url = "https://files.brokercheck.finra.org/firm/firm_{crd}.pdf"
    
    all_keys = REQUIRED_KEYS + [str(key) for key in OPTIONAL_KEY_DICT.keys()]
    
    def _log(msg, lvl=logging.WARNING):
        if debug:
            print("Potential FINRA BC parse error: " + str(msg))
        else:
            logger.log(lvl, "Potential FINRA BC parse error: " + str(msg))

    def preprocess_pdf_page_txt(page_txt):
        page_txt = page_txt.replace("User Guidance", "")
        page_txt = page_txt.replace("Sought:", "")
        page_txt = page_txt.replace("Sanction(s)/Relief", "Sanction(s)/Relief Sought:")
        page_txt = page_txt.replace("Does the order constitute a", "Final Order Flag:")
        page_txt = page_txt.replace("final order based on", "")
        page_txt = page_txt.replace("violations of any laws or", "")
        page_txt = page_txt.replace("regulations that prohibit", "")
        page_txt = page_txt.replace("fraudulent, manipulative, or", "")
        page_txt = page_txt.replace("deceptive conduct?", "")
        page_txt = re.sub("©.*\n", "", page_txt)
        page_txt = re.sub("All rights.*\n", "", page_txt)
        page_txt = re.sub("Report about .*\n", "", page_txt)
        page_txt = re.sub("www.finra.org/brokercheck.*\n", "", page_txt)
        page_txt = re.sub(" +", " ", page_txt).strip()
        
        for key in all_keys:
            if key not in IGNORE_ENSURE_NEW_LINE:
                page_txt = page_txt.replace(key + ":", "\n" + key + ":")
        
        return page_txt
        
    def filter_dict_list(dict_list):
        filtered_dict_list = []
        for idx, event_dict_list in enumerate(dict_list):
            _dict_to_parse = {}
            if len(event_dict_list) > 1:
                for entry in event_dict_list:
                    if entry.get('Reporting Source', "").lower() == 'regulator':
                        _dict_to_parse = entry
                        break
                    else:
                        _log("No 'Regulator' value found in 'Reporting Source' field for multi-entry event at index {idx} ..".format(idx))
            else:
                try:
                    _dict_to_parse = event_dict_list[0]
                except Exception:
                    _log("Empty entry occurred at index {idx} in parsed Broker Check report ..".format(idx))
                    _dict_to_parse = {}

            filtered_dict_list.append(_dict_to_parse)
        
        return filtered_dict_list

    remote_file = urlopen(pdf_url.format(crd=crd)).read()

    with io.BytesIO(remote_file) as memory_file:
        pdf = pdftotext.PDF(memory_file)

    start_idx = 0
    pdf_text = ' '.join([preprocess_pdf_page_txt(page) for page in list(pdf)[start_idx:]])

    result = regex.findall(r"(?<=Disclosure \d+ of \d.*\n)[\w\W]*?(?=Disclosure \d+ of \d+.*\n|$)", pdf_text)

    for idx, item in enumerate(result):
        for key in REQUIRED_KEYS:
            if key + ":" not in item:
                # _log("Key '{}' not in item #{}".format(key, idx))
                pass
    
    results_aggregated = [item for item in result if all([key in item for key in REQUIRED_KEYS])]

    result_reporting_source_separated = [[item for item in regex.findall(r"(?=Reporting Source:)[\w\W]*?(?=Reporting Source:|$)", r) if item.strip() != ""] for r in results_aggregated]

    disclosure_dict_pairs_list = []

    for idx_i, result_pair in enumerate(result_reporting_source_separated):
        _list = []
        for idx_j, r in enumerate(result_pair):
            data_dict = {}
            current_key = None

            for line in r.split('\n'):
                for key in all_keys:
                    if key + ':' in line:
                        # if we have already encounter a key
                        if current_key is not None:
                            data_dict[current_key] = val.strip()
                            val = ""

                        current_key = key
                        val = line[line.find(key + ":") + len(key + ":"):]

                        break
                else:
                    val += line

            data_dict[current_key] = val.strip()

            for key in all_keys:
                if key not in data_dict.keys():
                    default_val = OPTIONAL_KEY_DICT.get(key)
                    if default_val is None:
                        break
                    else:
                        data_dict[key] = default_val
            else:
                _list.append(data_dict)
        disclosure_dict_pairs_list.append(_list)
        
    filtered_dict_list = filter_dict_list(disclosure_dict_pairs_list)
    
    return filtered_dict_list

In [ ]:
results = parse_finra_broker_check_report("https://files.brokercheck.finra.org/firm/firm_11025.pdf")

In [ ]:
len(results)

In [ ]:
test_cases = [
    ("https://files.brokercheck.finra.org/firm/firm_877.pdf", 106, "Wedbush"),
    ("https://files.brokercheck.finra.org/firm/firm_19685.pdf", 5, "MUFG"),
    ("https://files.brokercheck.finra.org/firm/firm_19714.pdf", 90, "Barclays"),
    ("https://files.brokercheck.finra.org/firm/firm_79.pdf", 331, "J.P. Morgan"),
    ("https://files.brokercheck.finra.org/firm/firm_28269.pdf", 28, "Tradition Securities & Derivatives"),
    ("https://files.brokercheck.finra.org/firm/firm_33135.pdf", 26, "Lek Securities"),
    ("https://files.brokercheck.finra.org/firm/firm_6413.pdf", 150, "LPL Financial"),
    ("https://files.brokercheck.finra.org/firm/firm_2525.pdf", 266, "Deutsche Bank"),
    ("https://files.brokercheck.finra.org/firm/firm_7654.pdf", 265, "UBS")
]

cases_passed = 0

for case in test_cases:
    parsed_data = parse_finra_broker_check_report(case[0])
    
    try:
        assert len(parsed_data) == case[1], "Test case failed for entity '{}': expected {} records but got {}".format(case[2], case[1], len(parsed_data))
        cases_passed += 1
    except Exception as e:
        print(str(e))

if cases_passed == len(test_cases):
    print("All tests passed successfully")
else:
    print("Failed {} tests. Check the BrokerCheck reports to make sure the expected number of records is up-to-date.".format(len(test_cases) - cases_passed))


In [ ]:
import requests
import json

crd = "877"

url = "https://api.brokercheck.finra.org/firm/{crd}"

params = {
    "json.wrf": "angular.callbacks._0",
    "wt": "json",
}

r = requests.get(url=url.format(crd=crd), params=params)

In [ ]:
r = json.loads(r.text[len("/**/angular.callbacks._0("):-2])

In [ ]:
r = r['hits']['hits'][0]['_source']

In [ ]:
r

In [ ]:
import requests
import json
import time
import hashlib
from datetime import datetime

HASH_ENCODING = "utf-8"

DISCLOSURE_KEY_NAMES = ["Regulatory Event", "Civil Event", "Bond", "Arbitration"]

def get_finra_broker_check_summary(crd, feed_handle_owners, feed_handle_disclosures, feed_handle_meta):
    start_ts = time.time()
    
    api_url = "https://api.brokercheck.finra.org/firm/{crd}"
    summary_url = "https://brokercheck.finra.org/firm/summary/{crd}"

    params = {
        "json.wrf": "angular.callbacks._0",
        "wt": "json",
    }

    r = requests.get(url=api_url.format(crd=crd), params=params)

    data = json.loads(r.text[len("/**/angular.callbacks._0("):-2])
    data = json.loads(data['hits']['hits'][0]['_source']['content'])
    
    firm_name = data.get("basicInformation", {}).get("firmName", "")

    direct_owners_dict_list = data.get("directOwners", [])

    if len(direct_owners_dict_list):

        cached_direct_owner_hashes = feed_handle_owners.find({"crd": crd}, {"_id": True})
        cached_direct_owner_hashes = [item["_id"] for item in cached_direct_owner_hashes]

        for _dict in direct_owners_dict_list:

            # add broker check page link (if applicable)
            crd_str = _dict.get("crdNumber", "")
            if crd_str != "":
                _dict.update({
                    "url": summary_url.format(crd=crd_str)
                })

            # add unique '_id' to dict
            _hash = hashlib.sha1()
            _hash.update(firm_name.encode(HASH_ENCODING))
            _hash.update(_dict.get("legalName", "").encode(HASH_ENCODING))
            _hash.update(_dict.get("position", "").encode(HASH_ENCODING))
            hash_value = _hash.hexdigest()
            _dict.update({"_id": hash_value})
            
            # add 'new' flag to dict
            new_flag = (hash_value in cached_direct_owner_hashes)
            _dict.update({"new": new_flag})
            
            # add 'entity' to dict
            _dict.update({
                "entity": firm_name
            })
            
            # add 'crd' to dict
            _dict.update({
                "entity_crd": crd
            })
        
        # update API response with enriched data
        data.update({
            "directOwners": direct_owners_dict_list
        })
        
        # update database
        feed_handle_owners.delete_many({'entity_crd': crd})
        feed_handle_owners.insert_many(data["directOwners"])

    scraped_disclosure_count_dict = {item["disclosureType"]: item["disclosureCount"] for item in data.get("disclosures", [])}

    if len(scraped_disclosure_count_dict):

        cached_disclosure_count_dict = {item["type"]: item["count"] for item in feed_handle_disclosures.find({"crd": crd})}
        new_disclosure_dict_list = []

        for key in DISCLOSURE_KEY_NAMES:
            _dict = {}
            
            # add 'type' to dict
            _dict.update({
                "type": key
            })
            
            # add 'entity' to dict
            _dict.update({
                "entity": firm_name
            })

            # add 'crd' to dict
            _dict.update({
                "crd": crd
            })
            
            # add 'count' to dict
            new_value = scraped_disclosure_count_dict.get(key, 0)
            _dict.update({
                "count": new_value
            })

            # add 'change' to dict
            cached_value = cached_disclosure_count_dict.get(key, 0)
            _dict.update({
                "change": (new_value - cached_value)
            })
            
            new_disclosure_dict_list.append(_dict)
            
        # update API response with enriched data
        data.update({
            "disclosures": new_disclosure_dict_list
        })

        # update database
        feed_handle_disclosures.delete_many({'crd': crd})
        feed_handle_disclosures.insert_many(new_disclosure_dict_list) 

    days_elapsed_since_last_run = 0
    meta_dict = feed_handle_meta.find_one({"crd": crd})
    last_run_ts = meta_dict.get("last_run_ts") if meta_dict is not None else None

    if last_run_ts is not None:
        start_dt = datetime.fromtimestamp(start_ts)
        last_run_dt = datetime.fromtimestamp(last_run_ts)
        days_elapsed_since_last_run = (start_dt - last_run_dt).days

    # update API response with time elapsed
    data.update({
        "days_elapsed_since_last_run": days_elapsed_since_last_run,
        "last_run_ts": last_run_ts
    })
    
    # update API response with disclosure detail from PDF report
    try:
        disclosure_detail_dict_list = parse_finra_broker_check_report(crd)
    except Exception:
        disclosure_detail_dict_list = []
    
    data.update({
        "disclosure_detail": disclosure_detail_dict_list
    })

    # update last run date in database
    feed_handle_meta.delete_many({"crd": crd})
    feed_handle_meta.insert_one({"crd": crd, "last_run_ts": start_ts})

    return data

In [ ]:
from pymongo import MongoClient
handle_meta = MongoClient()["negative_news"]["broker_check_meta"]
handle_owners = MongoClient()["negative_news"]["broker_check_owners"]
handle_disclosures = MongoClient()["negative_news"]["broker_check_disclosures"]

In [ ]:
r = get_finra_broker_check_summary(877, handle_owners, handle_disclosures, handle_meta)

In [ ]:
r

In [ ]:
data = parse_finra_broker_check_report(877)

In [ ]:
data[:5]

In [ ]:
_dict_list = []
for idx_i, event_dict_list in enumerate(data):
    _dict_to_parse = {}
    if len(event_dict_list) > 1:
        for entry in event_dict_list:
            if entry.get('Reporting Source', "").lower() == 'regulator':
                _dict_to_parse = entry
                break
            else:
                logger.warning("Potential parse error (no regulatory source) occurred for entry {idx} in parsed Broker Check report ..".format(idx_i))
    else:
        try:
            _dict_to_parse = event_dict_list[0]
        except Exception:
            logger.warning("Potential parse error (empty entry) occurred for entry {idx} in parsed Broker Check report ..".format(idx_i))
            _dict_to_parse = {}

    _dict_list.append(_dict_to_parse)

In [ ]:
len(_dict_list)

In [ ]:
json.loads(data['content']).keys()

In [ ]:
meta_dict = None


In [ ]:
last_run_ts

# Deprecated FINRA Broker Check parsing code

In [ ]:
NEW_SECTION_FLAG = "Sanction Details"

ALLOWED_KEYS = [
    "Reporting Source",
    "Current Status",
    "Allegations",
    "Initiated By",
    "Date Initiated",
    "Docket/Case Number",
    "Principal Product Type",
    "Other Product Type(s)",
    "Principal Sanction(s)/Relief Sought",
    "Other Sanction(s)/Relief Sought",
    "Resolution Date",
    "Resolution",
    "Other Sanctions Ordered",
    "Sanctions Ordered",
    "Sanctions Details",
]


def clean_text(txt, preserve_new_lines=False):
    if not preserve_new_lines:
        txt = txt.replace("\n", " ")
    txt = txt.replace("\t", " ")
    txt = re.sub("\s+", " ", txt).strip()
    return txt

def parse_line(line_iter, results_list, data_dict=None, current_section=None, val=None, idx=None):

    if data_dict is None:
        data_dict = {}

    if val is None:
        val = ""
        
    if idx is None:
        idx = 0

    # try to get the next line;
    # if no line exists, return the data dictionary
    try:
        line = line_iter.__next__()
    except StopIteration:
        if current_section is not None:
            data_dict[current_section] = val
            data_dict['idx'] = idx
            results_list.append(data_dict)
        return results_list

    if ":" in line and any([key in line for key in ALLOWED_KEYS]):
        # if we've encountered a section already,
        # store the aggregated lines in the dictionary
        if current_section is not None:
            # if we hit the last key for the section,
            # append the current data dictionary and start
            # a new one
            if NEW_SECTION_FLAG in current_section:
                print("Starting new section. Length of data dict #{} is {} ..".format(idx, len(data_dict)))
                print("###########################")
                data_dict['idx'] = idx
                results_list.append(data_dict)
                data_dict = {}
                idx += 1
            # if the current section already exists in
            # the keys of the data dictionary, append the val
            # to that key
            elif current_section in data_dict.keys():
                print("'{}' section already exists in data dict #{}. Appending val ..".format(current_section, idx))
                data_dict[current_section] += val
            else:
                data_dict[current_section] = val

        # get the next section
        current_section = clean_text(re.findall(r'[A-Z].*(?=:)', line)[0]).lower()
        
#         print("section is now '" + current_section + "'")
        
        # reset the aggregrated lines to the value after the section name
        val = clean_text(" ".join(line.split(":")[1:]))
        
#         print("val after new section is '" + val + "'")
        
#         print("data_dict is: " + str(data_dict))
        
#         print("###")
    else:
        val += clean_text(line)

    return parse_line(line_iter, results_list, data_dict, current_section, val, idx)

# Disciplinary Actions

In [ ]:
import logging
from datetime import datetime

import requests
import dateparser

logger = logging.getLogger(__name__)

# constants
URL = "https://www.nyse.com/api/regulatory/disciplinary-actions/filter"
HEADERS = requests.utils.default_headers()
DOMAIN_TO_PREPEND = "nyse.com"


def get_nyse_disciplinary_actions(entity_name, year, max_results=20, offset=0, sort_type="up"):
    '''If bad request or empty response, returns None. 
       Otherwise, returns a dictionary with keys "num_found" and "results",
       where "results" is a list of dictionaries representing the extracted API results with
       the following keys:
           name {str} -- Name of the entity associated with the action
           action_type {str} -- Description of the action
           date {datetime.datetime} -- Date on which action occurred
           file_name {str} -- URL at which PDF can be found
    '''
    # make request
    params = {
        "market": "",
        "year": year,
        "sortType": sort_type,
        "filterToken": entity_name,
        "max": max_results,
        "offset": offset,
    }
    r = requests.get(url=URL, headers=HEADERS, params=params)

    if r.status_code == requests.codes.ALL_GOOD:
        _json = r.json()

        output = {}
        output['num_found'] = _json.get('totalCount')
        if output['num_found'] is None:
            raise Exception(f"Could not extract key 'totalCount' from the following JSON response: {_json}. Aborting ..")
        
        if output['num_found']:
            results = []
            for result in _json.get('results'):
                # convert timestamp integer to datetime
                timestamp = result.get('masterDateField')
                try:
                    date = datetime.fromtimestamp(timestamp / 1000)
                except Exception:
                    logger.error(f"Error occurred while parsing date from the following timestamp: {timestamp}. Skipping result ..")
                    continue

                # grab associated entity
                entity = result.get('name')
                if entity is None:
                    logger.error(f"Error occurred while parsing 'name' from result: {result}. Skipping result ..")
                    continue

                # grab action type
                action_type = result.get('actionType')
                if action_type is None:
                    logger.error(f"Error occurred while parsing 'actionType' from result: {result}. Skipping result ..")
                    continue

                # grab PDF file name
                file_name = DOMAIN_TO_PREPEND + result.get('fileName')
                if file_name is None:
                    logger.error(f"Error occurred while parsing 'fileName' from result: {result}. Skipping result ..")
                    continue
                    
                # grab doc reference number
                doc_ref = result.get('number', 'N/A')

                # append to results list
                results.append({
                    "date": date,
                    "entity": entity,
                    "action_type": action_type,
                    "file_name": file_name,
                    "doc_ref": doc_ref,
                })

            return results
            
        else:
            logger.info(f"Received empty response from NYSE API for entity '{entity_name}' ..")
    else:
        logger.error(f"Received bad request status from NYSE API: {r.status_code} ..")

In [ ]:
test = "hello {name}"
test.format(name="Will")

In [ ]:
import logging
from datetime import datetime, timezone
import re
import string

import requests
import dateparser
import pandas as pd
from bs4 import BeautifulSoup

logger = logging.getLogger("jupyter")

# Default headers to use for API requests
HEADERS = requests.utils.default_headers()

def clean_text(text):
    text = text.replace("\n", " ").replace("\t", " ").replace("\r", " ")
    text = re.sub(r" +", " ", text)
    text = text.strip()
    return text

def get_occ_disciplinary_actions(entity_name, date_str, max_results=100, offset=0, sort_type="StartDateDescending"):
    '''If bad request or empty response, returns None. 
       Otherwise, returns a dictionary with keys "num_found" and "results",
       where "results" is a list of dictionaries representing the extracted API results with
       the following keys:
           name {str} -- Name of the entity associated with the action
           action_type {str} -- Description of the action
           date {datetime.datetime} -- Date on which action occurred
           file_name {str} -- URL at which PDF can be found
           
       Parameters:
           entity_name {str} -- name of entity to search
           date_str {str} -- minimum date for which to include results (format="MM/DD/YYYY")
       
       Keyword Arguments:
           max_results {int} -- max number of items to display per page (default=100)
           offset {int} -- page number to display (default=0)
           sort_type {str} -- method by which to sort results (default="StartDateDescending")
           
       Sample API Response:
            [{'Amount': 0.0,
              'BankName': 'MUFG Bank, Ltd. Chicago Branch',
              'CharterNumber': 80140,
              'CityName': 'Chicago',
              'CompanyName': '',
              'CompleteDate': '2019-02-21T00:00:00-05:00',
              'DocketNumber': 'AA-EC-2019-7',
              'DocumentNumber': '2019-004',
              'HasPdf': True,
              'EnforcementTypeCode': 'C&D',
              'EnforcementTypeDescription': 'Cease & Desist Orders',
              'FirstName': '',
              'LastName': '',
              'StateAbbreviation': 'IL',
              'StateName': 'ILLINOIS            ',
              'TerminationDate': None,
              'TerminationDocumentNumber': '',
              'HasTerminationPdf': False},
            ]
    '''
    document_url = "https://www.occ.gov/static/enforcement-actions/ea{doc_ref}.pdf"
    url = "https://apps.occ.gov/EASearch/Search/ExportToJSON"
    params = {
        "Search": entity_name,
        "StartDateMinimum": date_str,
        "StartDateMaximum": "",
        "TerminationDateMinimum": "",
        "TerminationDateMaximum": "",
        "Category": "BankName",
        "Sort": sort_type,
        "AutoCompleteSelection": "",
        "CurrentPageIndex": offset,
        "ItemsPerPage": max_results,
        "ShowTerminatedActionsOnly": False,
        "ShowActiveOnly": False,
        "view": "Table",
        "InstitutionTypeEnums": "All",
    }

    r = requests.get(url=url, headers=HEADERS, params=params)

    results = []

    if r.status_code == requests.codes.ALL_GOOD:
        _json = r.json()

        for item in _json:
            # convert date str to datetime
            date_str = item.get('CompleteDate')
            try:
                date = dateparser.parse(date_str).replace(tzinfo=None)
            except Exception:
                logger.error(f"Error occurred while parsing date from the following date string: {date_str}. Skipping result ..")
                continue

            # grab associated entity
            entity = item.get('BankName')
            if entity is None:
                logger.error(f"Error occurred while parsing 'BankName' from result: {item}. Skipping result ..")
                continue

            # grab action type
            action_type = item.get('EnforcementTypeDescription')
            if action_type is None:
                logger.error(f"Error occurred while parsing 'EnforcementTypeDescription' from result: {item}. Skipping result ..")
                continue

            # grab document number
            doc_ref = item.get('DocumentNumber')
            if doc_ref is None:
                logger.error(f"Error occurred while parsing 'DocumentNumber' from result: {item}. Skipping result ..")
                continue
            else:
                file_name = document_url.format(doc_ref=doc_ref)

            # grab fine amount
            fine_amt = item.get('Amount')
            if fine_amt is None:
                logger.warning(f"Error occurred while parsing 'Amount' from result: {item}. Default to zero ..")
                fine_amt = 0.0

            # append to results list
            results.append({
                "date": date,
                "entity": entity,
                "action_type": action_type,
                "file_name": file_name,
                "doc_ref": doc_ref,
                "fine_amt": fine_amt,
            })
    else:
        logger.error(f"Received bad request status from NYSE API: {r.status_code} ..")

    return results

In [ ]:
get_occ_disciplinary_actions("Wells", "01/01/2018")

In [ ]:
import string
import re
import logging

import requests
import pandas as pd
from bs4 import BeautifulSoup
import dateparser

logger = logging.getLogger(__name__)


def get_finra_disciplinary_actions(entity_name, date_str):
    
    expected_col_names = [
        'case_id',
        'case_summary',
        'document_type',
        'firms_individuals',
        'action_date',
        'file_name',
    ]
    
    def get_href(soup, text_to_search):
        for link in soup.findAll("a"):
            if link.text == text_to_search:
                return link.get("href")
        return ""

    url = "http://www.finra.org//industry/disciplinary-actions/finra-disciplinary-actions-online"
    headers = requests.utils.default_headers()
    df_master = pd.DataFrame()
    page = 0

    while True:
        params = {
            "firms": entity_name,
            "field_core_official_dt_value[min][date]": date_str,
            "field_core_official_dt_value[max][date]": "03/07/2019",
            "page": page,
        }
        r = requests.get(url=url, headers=headers, params=params)

        # try to extract table
        try:
            df = pd.read_html(r.text)[0]
        except Exception:
            break

        # each column name gets copied into each of its cells
        # this function removes the column names
        df = df.apply(lambda series: series.str.replace(series.name, ""))

        # strip all white space from column strings
        df = df.apply(lambda series: series.str.strip())

        # extract all links
        soup = BeautifulSoup(r.text, "lxml")
        df["File Name"] = df["Case ID"].apply(lambda val: get_href(soup, val))

        df_master = pd.concat([df_master, df], ignore_index=True)

        page += 1
    
    if len(df_master):
        # rename all columns
        df_master.rename(columns=lambda x: re.sub("(?<=[a-z])([ {}])*(?=[A-Z])".format(string.punctuation), "_", x).lower(), inplace=True)

        # convert to dates to datetime
        df_master['action_date'] = df_master['action_date'].apply(dateparser.parse)
        df_master['action_date'] = df_master['action_date'].dt.to_pydatetime()
        
        # if the column names do not match what we expect to return,
        # log an error message and return an empty list
        if any([col_name not in df_master.columns for col_name in expected_col_names]):
            logger.error("Not all expected column names found in extracted FINRA column names: {}. Returning empty list ..".format(', '.join(df_master.columns)))
            return []

        # convert to list of dictionaries
        return df_master.to_dict("records")
    else:
        return []

In [ ]:
get_finra_disciplinary_actions("Deutsche", "01/01/2017")

In [ ]:
def get_nydfs_disciplinary_actions(entity_name_list):
    COL_IDX_DATE = 0
    COL_IDX_TITLE = 1
    document_url = "https://dfs.ny.gov{href}"
    results = []

    r = requests.get("https://www.dfs.ny.gov/industry_guidance/enforcement_actions_lfs")
    
    if r.status_code == requests.codes.ALL_GOOD:
        soup = BeautifulSoup(r.text)
        table = soup.find('table')
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')

        for row in rows:
            data = {}
            cols = row.find_all('td')
            text_content = [ele.text for ele in cols if ele]

            if not len(text_content):
                continue

            data['doc_title'] = text_content[COL_IDX_TITLE]

            for name in entity_name_list:
                if name.lower() in text_content[COL_IDX_TITLE].lower():
                    data['entity'] = name.upper()
                    break
            else:
                continue

            data['action_date'] = dateparser.parse(text_content[COL_IDX_DATE])

            links = row.find_all("a")
            href_list = []
            for l in links:
                href = l.get("href")
                if href is not None:
                    href_list.append(document_url.format(href=href))

            data['file_names'] = href_list

            results.append(data)
    else:
        logger.error(f"Received bad request status from NYDFS API: {r.status_code}. Returning empty list ..")
        return []
    
    return results

In [ ]:
get_nydfs_disciplinary_actions(["Barclays"])

In [ ]:
import requests
r = requests.get("https://www.sec.gov/litigation/apdocuments/ap-open-fileno-asc.xml")

In [ ]:
r.apparent_encoding

In [ ]:
import xml.etree.ElementTree as et

In [ ]:
tree = et.parse(r.content)

In [ ]:
feed = feedparser.parse("https://www.sec.gov/rss/litigation/admin.xml")
feed['entries']

In [ ]:
from datetime import datetime

import dateparser
import feedparser

def get_sec_litigation_releases(entity_name_list):
    url = "https://www.sec.gov/rss/litigation/admin.xml"
    
    if not isinstance(entity_name_list, list):
        raise ValueError(f"'entity_name_list' should be of type list, but got type {type(entity_name_list)}")
    
    try:
        entries = feedparser.parse(url)["entries"]
    except Exception as e:
        logger.warning(f"Exception occurred while retrieving entries from SEC litigation feed: {str(e)}")
        return []

    entries = [item for item in entries if any(name in item["title"] for name in entity_name_list)]
    
    results = []
    for item in entries:
        entity = item.get("title", "")
        if entity is None:
            logger.error(f"Error occurred while parsing 'title' from result: {item}. Skipping result ..")
            continue
        
        action_type = item.get("summary", "")
        if action_type is None:
            logger.error(f"Error occurred while parsing 'summary' from result: {item}. Skipping result ..")
            continue
        
        doc_ref = item.get("id", "N/A")
        if doc_ref is None:
            logger.error(f"Error occurred while parsing 'doc_ref' from result: {item}. Defaulting to 'N/A' ..")
        
        file_name = item.get("link")
        if file_name is None:
            logger.error(f"Error occurred while parsing 'file_name' from result: {item}. Skipping result ..")
            continue

        # date
        time_struct = item.get('published_parsed')
        if time_struct is not None:
            date = datetime(year=time_struct.tm_year, month=time_struct.tm_mon, day=time_struct.tm_mday)
        else:
            time_str = item.get('published')
            if time_str is not None:
                date = dateparser.parse(time_str)

        results.append({
            "date": date,
            "entity": entity,
            "action_type": action_type,
            "file_name": file_name,
            "doc_ref": doc_ref,
        })

    return results

In [ ]:
get_sec_litigation_releases(["Wells Fargo"])

In [20]:
from nltk.tokenize import sent_tokenize
for item in data:
    item['sent_tokenized'] = sent_tokenize(item['content'])

In [24]:
def bin_sentiment(sentiment_score):
    if sentiment_score < -0.05:
        return -1
    elif sentiment_score > 0.05:
        return 1
    else:
        return 0

In [25]:
for item in data:
    item['sentiment_class_list'] = [bin_sentiment(score) for score in item['sentiment_list']]

In [27]:
data[0]

{'_id': ObjectId('5c8938fbb1a779022d863f0e'),
 'all_search_terms': ['',
  'Fined',
  'Violation',
  'Penalty',
  'Fraud',
  'Fine',
  'Sanction'],
 'classification_tuple_lists': [[2, 'regulatory_reference'],
  [7, 'mgmt_change'],
  [9, 'statutory_disqual'],
  [7, 'monetary_fine_major']],
 'content': 'Wells Fargo will pay $575 million in a settlement with attorneys general from all 50 states and the District of Columbia that are investigating its banking practices, which have included phony accounts and manipulative sales practices. The agreement, announced on Friday by the San Francisco-based bank and Iowa\'s attorney general, settles a range of allegations that have roiled the bank and its customers for years, including the 2016 revelation that employees had opened millions of\xa0fake bank accounts\xa0to meet sales quotas. The settlement also covers a host of other improper practices involving insurance, auto loans, mortgages and financing.\xa0 "This agreement underscores our serious 

In [38]:
sentence_sentiment_tuples = [(sentence, _class) for item in data for sentence, _class in zip(item['sent_tokenized'], item['sentiment_class_list'])]

In [43]:
len(sentence_sentiment_tuples)

213996

In [42]:
sentence_sentiment_tuples[0]

('Wells Fargo will pay $575 million in a settlement with attorneys general from all 50 states and the District of Columbia that are investigating its banking practices, which have included phony accounts and manipulative sales practices.',
 1)

In [89]:
'''Demo for sentiment classifier using Pymagnitude leveraging Keras.

Usage: sentiment_classifier_demo.py --conn_str mongodb://192.168.0.2:27017 --db_name negative_news --coll_name processed_results \
            --magnitude_path /path/to/mag/file
'''
from math import ceil
import argparse
import time

from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import GaussianNoise, LSTM, Bidirectional, Dropout, Dense
from keras.optimizers import Adam
from keras.models import model_from_json
from pymagnitude import Magnitude, MagnitudeUtils
from pymongo import MongoClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# The maximum number of words the sequence model will consider
MAX_WORDS = 30
# Deviation of noise for Gaussian Noise applied to the embeddings
STD_DEV = 0.01
# The number of hidden units from the LSTM
HIDDEN_UNITS = 100
# The ratio to dropout
DROPOUT_RATIO = 0.8
# The number of examples per train/validation step
BATCH_SIZE = 100
# The number of times to repeat through all of the training data
EPOCHS = 200
# The learning rate for the optimizer
LEARNING_RATE = 0.01
# The number of output classes in the sample data
NUM_OUTPUT_CLASSES = 3
# Lower bound VADER sentiment threshold for neutral
SENTIMENT_NEUTRAL_LB = -0.05
# Upper bound VADER sentiment threshold for neutral
SENTIMENT_NEUTRAL_UB = 0.05
# Percentage of observations included in test sample
TEST_SIZE = 0.25


def load_data(conn_str, mongo_db_str, mongo_coll_str):
    '''Loads data from MongoDB and returns a cursor'''
    client = MongoClient(conn_str)
    db = client[mongo_db_str]
    coll = db[mongo_coll_str]
    return coll.find({})


def preprocess_data(cursor):
    '''Preprocesses all data associated with a MongoDB cursor until it is exhausted.
        Output data is a list of tuples where each tuple is of the following form:
            (word token list, sentiment class)
        'Word token list' represents the word tokens of a sentence while 
        'sentiment class' represents the VADER sentiment classification for the sentence [-1 = negative, 0 = neutral, 1 = positive ].
    '''
    def bin_sentiment(sentiment_score):
        '''Bins continuous sentiment score into discrete buckets'''
        if sentiment_score < SENTIMENT_NEUTRAL_LB:
            return -1
        elif sentiment_score > SENTIMENT_NEUTRAL_UB:
            return 1
        else:
            return 0

    data = []

    for item in cursor:
        sentence_tokens = sent_tokenize(item['content'])

        analyzer = SentimentIntensityAnalyzer()
        sentiment_class_list = [bin_sentiment(analyzer.polarity_scores(sentence[:MAX_WORDS])["compound"]) for sentence in sentence_tokens]

        for idx, sentence in enumerate(sentence_tokens):
            data.append((sentence[:MAX_WORDS], sentiment_class_list[idx]))

    return data


def build_model():
    '''Builds Keras model'''
    model = Sequential()
    model.add(GaussianNoise(STD_DEV, input_shape=(MAX_WORDS, vectors.dim)))
    model.add(Bidirectional(LSTM(HIDDEN_UNITS, activation='tanh'), merge_mode='concat'))
    model.add(Dropout(DROPOUT_RATIO))
    model.add(Dense(NUM_OUTPUT_CLASSES, activation='softmax'))
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(lr=LEARNING_RATE),
        metrics=['categorical_accuracy']
    )
    return model


def save_model(model, model_output_name):
    '''Saves Keras model JSON and weights'''
    model_json = model.to_json()
    with open("{}.json".format(model_output_name), "w") as f:
        f.write(model_json)

    model.save_weights("{}.h5".format(model_output_name))


def load_model(model_name):
    '''Loads Keras model JSON and weights; returns Keras model object'''
    # load model json
    with open('{}.json'.format(model_name), 'r') as f:
        model_json = f.read()

    # create model from json
    model = model_from_json(model_json)

    # load weights into new model
    model.load_weights("{}.h5".format(model_name))

    return model


def train(model, data, magnitude_path):
    '''Initiates training for Keras model'''
    X, y = zip(*data)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=101)

    training_batches = MagnitudeUtils.batchify(X_train, y_train, BATCH_SIZE) # Split the training data into batches
    num_batches_per_epoch_train = int(ceil(len(X_train)/float(BATCH_SIZE)))
    test_batches = MagnitudeUtils.batchify(X_test, y_test, BATCH_SIZE)  # Split the test data into batches
    num_batches_per_epoch_test = int(ceil(len(X_test)/float(BATCH_SIZE)))

    vectors = Magnitude(magnitude_path)

    # Generates batches of the transformed training data
    train_batch_generator = (
      (
        vectors.query(X_train_batch), # Magnitude will handle converting the 2D array of text into the 3D word vector representations
        MagnitudeUtils.to_categorical(y_train_batch, NUM_OUTPUT_CLASSES) # Magnitude will handle converting the class labels into one-hot encodings
      ) for X_train_batch, y_train_batch in training_batches
    )

    # Generates batches of the transformed test data
    test_batch_generator = (
      (
        vectors.query(X_test_batch), # Magnitude will handle converting the 2D array of text into the 3D word vector representations
        MagnitudeUtils.to_categorical(y_test_batch, NUM_OUTPUT_CLASSES) # Magnitude will handle converting the class labels into one-hot encodings
      ) for X_test_batch, y_test_batch in test_batches
    )

    # Start training
    model.fit_generator(
        generator=train_batch_generator,
        steps_per_epoch=num_batches_per_epoch_train,
        validation_data=test_batch_generator,
        validation_steps=num_batches_per_epoch_test,
        epochs=EPOCHS,
    )


def get_parser():
    '''Builds CLI argument parser for this program'''
    parser = argparse.ArgumentParser(description='Demo Sentiment Classifier')
    parser.add_argument('--conn_str', type=str, help='MongoDB connection string')
    parser.add_argument('--db_name', type=str, help='MongoDB database name')
    parser.add_argument('--coll_name', type=str, help='MongoDB collection name')
    parser.add_argument('--magnitude_path', type=str, help='Path to Magnitude file')
    parser.add_argument('--data_path', type=str, default="", help="Path to preprocessed data pickle (optional)")
    parser.add_argument('--model_load_path', type=str, default="", help="Path to pre-trained model")
    parser.add_argument('--output_path', type=str, default='model', help='Path for model output (optional)')
    return parser


def main(params):
    if params.data_path != "":
        print("Loading data (conn_str={params.conn_str}, db_name={params.db_name}, coll_name={params.coll_name}) ..".format(
                params.conn_str, params.db_name, params.coll_name
            )
        )
        tock = time.perf_counter()
        cursor = load_data(params.conn_str, params.db_name, params.coll_name)
        tick = time.perf_counter()
        print("Successfully loaded data. Took {}s ..".format(round(tick - tock, 2)))

        print("Preprocessing data ..")
        tock = time.perf_counter()
        data = preprocess_data(cursor)
        tick = time.perf_counter()
        print("Successfully preprocessed data. Took {}s ..".format(round(tick - tock, 2)))
    else:
        raise Exception("Loading data from pickle not yet supported")

    if params.model_load_path != "":
        model = build_model()
    else:
        model = load_model(param.model_load_path)

    train(model, data)

    save_model(model, params.output_path)
    print("Saved model JSON and weights at following path: {}".format(params.output_path))


if __name__ == '__main__':
    parser = get_parser()
    params = parser.parse_args()
    main(params)

In [92]:
model

In [51]:
vectors = Magnitude("/home/tempsudo/transfer_learning_experiment/embeddings/crawl-300d-2M.magnitude")

MAX_WORDS = 30 # The maximum number of words the sequence model will consider
STD_DEV = 0.01 # Deviation of noise for Gaussian Noise applied to the embeddings
HIDDEN_UNITS = 100 # The number of hidden units from the LSTM
DROPOUT_RATIO = .8 # The ratio to dropout
BATCH_SIZE = 100 # The number of examples per train/validation step
EPOCHS = 200 # The number of times to repeat through all of the training data
LEARNING_RATE = .01 # The learning rate for the optimizer

num_outputs = 3

model = Sequential()
model.add(GaussianNoise(STD_DEV, input_shape=(MAX_WORDS, vectors.dim)))
model.add(Bidirectional(LSTM(HIDDEN_UNITS, activation='tanh'), merge_mode='concat'))
model.add(Dropout(DROPOUT_RATIO))
model.add(Dense(num_outputs, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=LEARNING_RATE),
    metrics=['categorical_accuracy'])

In [76]:
training_batches = MagnitudeUtils.batchify(X_train, y_train, BATCH_SIZE) # Split the training data into batches
num_batches_per_epoch_train = int(ceil(num_training/float(BATCH_SIZE)))
test_batches = MagnitudeUtils.batchify(X_test, y_test, BATCH_SIZE)  # Split the test data into batches
num_batches_per_epoch_test = int(ceil(num_test/float(BATCH_SIZE)))

# Generates batches of the transformed training data
train_batch_generator = (
  (
    vectors.query(X_train_batch), # Magnitude will handle converting the 2D array of text into the 3D word vector representations!
    MagnitudeUtils.to_categorical(y_train_batch, num_outputs) # Magnitude will handle converting the class labels into one-hot encodings!
  ) for X_train_batch, y_train_batch in training_batches
)

# Generates batches of the transformed test data
test_batch_generator = (
  (
    vectors.query(X_test_batch), # Magnitude will handle converting the 2D array of text into the 3D word vector representations!
    MagnitudeUtils.to_categorical(y_test_batch, num_outputs) # Magnitude will handle converting the class labels into one-hot encodings!
  ) for X_test_batch, y_test_batch in test_batches
)

# Start training
from keras.utils import np_utils
model.fit_generator(
    generator = train_batch_generator,
    steps_per_epoch = num_batches_per_epoch_train,
    validation_data = test_batch_generator,
    validation_steps = num_batches_per_epoch_test,
    epochs = EPOCHS,
)

Epoch 1/200
1605/1605 [==============================] - 2392s 1s/step - loss: 0.5838 - categorical_accuracy: 0.7686 - val_loss: 0.3345 - val_categorical_accuracy: 0.8828
Epoch 2/200
1605/1605 [==============================] - 2298s 1s/step - loss: 0.3124 - categorical_accuracy: 0.8942 - val_loss: 0.2187 - val_categorical_accuracy: 0.9295
Epoch 3/200
  16/1605 [..............................] - ETA: 52:15 - loss: 0.2441 - categorical_accuracy: 0.9206

KeyboardInterrupt: 

In [87]:
save_model(model, "test_1")